## String

**Q 1:** Given a string, find the first non-repeating character. (Find the first character which appears only once in the string)  
**Answer:** We can maintain a map with key being the key and value being the count.

In [1]:
public char firstNonRepeatingChar(String str) {
    Map<Character, Integer> map = new HashMap<>();
    for(int i = 0; i < str.length(); i++) {
        Integer count =  map.getOrDefault(str.charAt(i), 0);
        map.put(str.charAt(i), ++count);
    }

    for(int i = 0; i < str.length(); i++) {
        if (map.get(str.charAt(i)) == 1) {
            return str.charAt(i);
        }
    }
    
    return 0;
}

System.out.println(firstNonRepeatingChar("RoundRobin"));

u


**Q 2:** Given two strings return if they are isomorphic or not. Two strings are isomorphic if we have a 1-1 mapping between characters of both strings. For example, `foo` and `abb` are isomorphic with mapping `f:a, o:b`. Length of both strings will always be equal.  
**Answer:** 

In [2]:
public boolean isomorphic(String first, String second) {
    Map<Character, Character> secondToFirst = new HashMap<>();
    Map<Character, Character> firstToSecond = new HashMap<>();
    for (int i=0; i<first.length(); i++) {
        secondToFirst.put(second.charAt(i), first.charAt(i));
        firstToSecond.put(first.charAt(i), second.charAt(i));
    }

    return isomorphic(first, second, secondToFirst) && isomorphic(second, first, firstToSecond);
}

private boolean isomorphic(String first, String second, Map<Character, Character> map) {
    char[] secondArray = second.toCharArray();
    for (int i = 0; i < second.length(); i++) {
        secondArray[i] = map.get(second.charAt(i));
    }

    return first.equals(new String(secondArray));
}

System.out.println(isomorphic("foo", "bar"));
System.out.println(isomorphic("foo", "baa"));

false
true


Maintain two maps : one contains the mapping of characters from 1st string to the second string. The other map maintains all the characters in second string which have already been mapped.

In [3]:
public boolean isomorphic2(String first, String second) {
    Map<Character, Character> map = new HashMap<>();
    Set<Character> used = new HashSet<>();

    for (int i = 0; i < first.length(); i++) {
        if (!map.containsKey(first.charAt(i))) {
            if (!used.contains(second.charAt(i))) {
                map.put(first.charAt(i), second.charAt(i));
                used.add(second.charAt(i));
            } else {
                return false;
            }
        } else {
            if (!map.get(first.charAt(i)).equals(second.charAt(i))) {
                return false;
            }
        }
    }

    return true;
}

System.out.println(isomorphic2("foo", "bar"));
System.out.println(isomorphic2("foo", "baa"));

false
true


**Q 3:** Given two strings. Is the second string substring of the first one? If first string is `CAANADA` and the second string is `ANA`, the answer would be true.  
**Answer:** Use KMP or Z-Algorithm. Or use rolling hash algorithm Rabin-Karp. Let the longer string be `T` (text) and the string to search is `P` (pattern).  
A naive approach is that for each index of `T` try matching character by character in `P`. This will take $O(n*m)$ time complexity.

**KMP:** avoids re-checking characters that we already know match. When a mismatch happens:
- Instead of moving the pattern by one position,
- We use information from a preprocessed table (called the LPS array) to skip ahead intelligently.
Time complexity is $O(m + n)$

In [ ]:
// KMP
public boolean contains(String text, String pattern) {
    if (pattern == null || text == null) return false;
    if (pattern.length() == 0) return true; // empty pattern always matches
    if (pattern.length() > text.length()) return false;

    int[] lps = computeLPSArray(pattern);

    int i = 0; // index for text
    int j = 0; // index for pattern

    while (i < text.length()) {
        if (pattern.charAt(j) == text.charAt(i)) {
            i++;
            j++;
        }
        if (j == pattern.length()) {
            return true; // full pattern matched
        } else if (i < text.length() && pattern.charAt(j) != text.charAt(i)) {
            if (j != 0) {
                j = lps[j - 1];
            } else {
                i++;
            }
        }
    }
    return false; // no match found
}

private int[] computeLPSArray(String pattern) {
    int m = pattern.length();
    int[] lps = new int[m];
    int len = 0;
    int i = 1;

    while (i < m) {
        if (pattern.charAt(i) == pattern.charAt(len)) {
            len++;
            lps[i] = len;
            i++;
        } else {
            if (len != 0) {
                len = lps[len - 1];
            } else {
                lps[i] = 0;
                i++;
            }
        }
    }
    return lps;
}

**Z Algorithm:** computes a Z-array for a string. `Z[i]` is length of the longest substring starting at `i` that is also a prefix of the string. This also has time complexity $O(m + n)$.

In [ ]:
// Z Algorithm
public boolean contains(String text, String pattern) {
    if (pattern == null || text == null) return false;
    if (pattern.length() == 0) return true; // empty pattern always matches
    if (pattern.length() > text.length()) return false;

    // Create combined string: pattern + "$" + text
    String concat = pattern + "$" + text;
    int[] Z = computeZArray(concat);

    int patternLength = pattern.length();

    for (int zValue : Z) {
        if (zValue == patternLength) {
            return true; // full pattern matched
        }
    }
    return false;
}

private int[] computeZArray(String s) {
    int n = s.length();
    int[] Z = new int[n];
    int L = 0, R = 0;

    for (int i = 1; i < n; i++) {
        if (i <= R) {
            Z[i] = Math.min(R - i + 1, Z[i - L]);
        }
        while (i + Z[i] < n && s.charAt(Z[i]) == s.charAt(i + Z[i])) {
            Z[i]++;
        }
        if (i + Z[i] - 1 > R) {
            L = i;
            R = i + Z[i] - 1;
        }
    }
    return Z;
}

**Rabin Karp:** uses hash to speed up string search. Has $O(m + n)$ complexity.

In [ ]:
// Rabin Karp
public boolean contains(String text, String pattern) {
    if (pattern == null || text == null) return false;
    if (pattern.length() == 0) return true; // empty pattern always matches
    if (pattern.length() > text.length()) return false;
    
    long patternHash = hash(pattern);
    long textHash = hash(text.substring(0, pattern.length()));
    if (patternHash == textHash) {
        return true;
    }

    for (int i = pattern.length(); i < text.length(); i++) {
        textHash = (textHash - (long) (text.charAt(i - pattern.length()) * Math.pow(31, pattern.length() - 1))) * 31 + text.charAt(i);
        // Compare hash and substring (to prevent collision related match)
        if (textHash == patternHash && text.substring(i - pattern.length() + 1, i + 1).equals(pattern)) {
            return true;
        }
    }

    return false;
}

private long hash(String input) {
    long result = 0;
    for (int i = 0; i < input.length(); i++) {
        result += (long) (input.charAt(i) * Math.pow(31, input.length() - i - 1));
    }

    return result;
}

**Q 4** Reverse a string word by word. Reduce multiple spaces to single.  
**Answer** Make use of a stack. Put all words in stack and then pop from stack.

In [4]:
public String reverseWords(String input) {
    Stack<String> stack = new Stack<>();

    StringBuilder temp = new StringBuilder();
    for (int i = 0; i < input.length(); i++) {
        if (input.charAt(i) == ' ') {
            stack.push(temp.toString());
            stack.push(" ");
            temp = new StringBuilder();
        } else if (i == input.length() - 1) {
            stack.push(temp.toString());
        } else {
            temp.append(input.charAt(i));
        }
    }

    StringBuilder result = new StringBuilder();
    while (!stack.isEmpty()) {
        result.append(stack.pop());
    }

    return result.toString();
}

System.out.println(reverseWords("the sky is blue"));

blu is sky the


If we want constant space,

In [5]:
public String reverseWords2(String input) {
    int i = 0, j = 0;
    char[] inputArray = input.toCharArray();
    for (; j <= inputArray.length; j++) {
        if (j == inputArray.length || inputArray[j] == ' ') {
            // Reverse characters from i to j - 1
            int a = i, b = j - 1;
            while (a < b) {
                char temp = inputArray[a];
                inputArray[a] = inputArray[b];
                inputArray[b] = temp;
                a++;
                b--;
            }

            i = j + 1;
        }
    }

    int a = 0, b = inputArray.length - 1;
    while (a < b) {
        char temp = inputArray[a];
        inputArray[a] = inputArray[b];
        inputArray[b] = temp;
        a++;
        b--;
    }

    return new String(inputArray);
}

System.out.println(reverseWords2("the sky is blue"));

blue is sky the


**Q 5:** Convert a string to integer. The string can contain characters other than digits as well. For example, if the string is `A96 B1` return `96` .  
**Answer:**

In [6]:
public int firstNumberInString(String input) {
    int i = -1;
    StringBuilder sb = new StringBuilder();
    for (int j = 0; j < input.length(); j++) {
        if (((int) input.charAt(j) <= 57 && (int) input.charAt(j) >= 48) || // number between '0' and '9'
            // input[j] is '-' and input[j+1] is a number between '0' and '9'
            (j + 1 < input.length() && input.charAt(j) == '-' && ((int) input.charAt(j + 1) <= 57 && (int) input.charAt(j + 1) >= 48))) {
            sb.append(input.charAt(j));
            i = j;
            break;
        }
    }

    for (int j = i + 1; j < input.length(); j++) {
        if ((int) input.charAt(j) <= 57 && (int) input.charAt(j) >= 48) {
            sb.append(input.charAt(j));
        } else {
            break;
        }
    }

    return Integer.parseInt(sb.toString());
}
        
System.out.println(firstNumberInString(" --1*5*6ol5"));

-1


**Q 6:** Given a string, return the length longest substring with unique characters.  
**Answer:** Maintain a map which contains index of occurance of a character

In [7]:
public int longestStringUniqueChars(String input) {
    int max = Integer.MIN_VALUE;
    Map<Character, Integer> map = new HashMap<>();

    int i = 0, j = 0;
    while (j < input.length()) {
        if (map.containsKey(input.charAt(j))) {
            i = Math.max(i, map.get(input.charAt(j)) + 1);
        }

        map.put(input.charAt(j), j);

        if (j - i + 1 > max) {
            max = j - i + 1;
        }

        j++;
    }

    return max;
}

System.out.println(longestStringUniqueChars("abdeefgabef"));

5


**Q 7:** Given a string with `a` and `b` as the only characters. We can either convert `a` to `b` or reverse $K$ times. Find the length of the longest substring with same characters.  
**Answer:** Consider `a` as bad character and try to replace it (max K times). Slide the window when replacements > $K$. Next consider `b` as bad character and do the same.